In [35]:
import sqlite3 
import pandas as pd 
import requests
import json  
from datetime import datetime
import secrets
import telegram #pip install python-telegram-bot --upgrade
import time
import math

def connect_create_table(name_db='db.db'):
    conn = sqlite3.connect(name_db)  # You can create a new database by changing the name within the quotes
    c = conn.cursor() # The database will be saved in the location where your 'py' file is saved

    # Create table - CLIENTS
    c.execute('''CREATE TABLE if not exists POSITIONS
                 ([id] INTEGER PRIMARY KEY,
                 [date] DATETIME,
                 [encryptedUid] TEXT,
                 [nick] TEXT,
                 [symbol] TEXT,
                 [status] TEXT,
                 [entryPrice] FLOAT,
                 [markPrice] FLOAT,
                 [pnl] FLOAT,
                 [roe] FLOAT,
                 [amount] FLOAT,
                 [updateTimeStamp] TEXT,
                 [request_id] TEXT
                 )''')
    conn.commit()

connect_create_table()

In [36]:
def insert_in_table(trader_info, request_id, vector, status):
    encryptedUid = trader_info[0]
    nick         = trader_info[1]
    
    try:
        sqliteConnection = sqlite3.connect('db.db')
        cursor = sqliteConnection.cursor()
        
        symbol      = vector[0]
        entryPrice  = vector[1]
        markPrice   = vector[2]
        pnl         = vector[3]
        row         = vector[4]
        amount      = vector[5]
        updateTimestamp = time.time()
        date        = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

        # Need to add more info of student (such as the txns)
        sqlite_insert_with_param = """INSERT INTO POSITIONS 
                          (date, encryptedUid, nick, symbol, status, entryPrice, markPrice, pnl, roe, amount, updateTimeStamp, request_id) 
                          VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?,?,?);"""

        data_tuple = (date, encryptedUid, nick, symbol, status, entryPrice, markPrice, pnl, row, amount, updateTimestamp,request_id) 
        cursor.execute(sqlite_insert_with_param, data_tuple)
        sqliteConnection.commit()

        cursor.close()

    except sqlite3.Error as error:
        print("Failed to insert Python variable into sqlite table", error)
    finally:
        if sqliteConnection:
            sqliteConnection.close()

In [37]:
def do_query(db="db.db", query="SELECT * FROM POSITIONS"):    
    con = sqlite3.connect(db)
    df = pd.read_sql_query(query, con)
    
    return df

In [38]:
def get_positions(trader):
    endpoint = "https://www.binance.com/bapi/futures/v1/public/future/leaderboard/getOtherPosition"
    params   =  {"encryptedUid":trader,  "tradeType":"PERPETUAL"}
    headers  =  {"content-type":"application/json;charset=UTF-8"}
    response = requests.post(endpoint, json=params, headers=headers) #Add proxies = proxyDict # Attention to JSON = PARAMS
    result   = response.json()
    
    return result

In [39]:
def send_message(text, chat_id = "-1001455341077"):
    my_bot  = telegram.Bot("814569733:AAGmMk_P4p8EdLkj-fj2fVziyByxHIcJpm0")
    my_bot.send_message(chat_id=chat_id, text=text, parse_mode = "Markdown")

In [40]:
def send_message_discord(text):
    #Webhook of my channel. Click on edit channel --> Webhooks --> Creates webhook
    webhook = "https://discord.com/api/webhooks/892399337731133500/_IE9zchjLp3lRrImBt8sp_YgrI8A9Kan1Vc6wv7eY7vvkVNEAGF-YkVkiUF4pKxT5J3R"
    data    = {"content": text}
    response = requests.post(webhook, json=data)

In [41]:
def create_message(trader, row, status):
    trade_type = "NONE"
    
    if row["amount_new"] > 0:
        side = "long"
        if row["difference"] > 0:
            trade_type = "BUY"
        else:
            trade_type = "SELL"

    if row["amount_new"] < 0:
        side = "short"
        if row["difference"] > 0:
            trade_type = "BUY"                        
        else:
            trade_type = "SELL" 
    
    current_position = str(round(row["amount_new"],4))
    old_position     = str(round(row["amount_old"],4))
    price            = str(round(row["markPrice"],4))
    difference       = str(abs(row["difference"]))
    symbol           = row["symbol"]
    date             = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
       
    message =  "💰" + trader + " " + date + "\nPosition change from " + old_position + symbol + " to " + current_position + symbol + ".\n" + trade_type + " " + difference + symbol + " at " + price + "$" + "\nStatus:" + status 
    return message 

In [46]:
names    = {"CCF3E0CB0AAD54D9D6B4CEC5E3E741D2":"TraderT", 
            "49A7275656A7ABF56830126ACC619FEB":"ADnefina", 
            "A532C4316C00206168F795EDFBB3E164":"ClickHereNow", 
            "D3AFE978B3F0CD58489BC27B35906769":"CheerWindCard",
            "FB23E1A8B7E2944FAAEC6219BBDF8243":"BananaMan1", 
            "F318AE1B1BB3AEF4EDBD36E5AE250CD1":"BigPiePlayer", 
            "2F9D01E80429F97670415A5A5DDD9405":"CryptoPinoy"}

#names = { "D3AFE978B3F0CD58489BC27B35906769":"CheerWindCard"}

status = "OPEN"
error_download = False
traders  = list(names.keys())
keys     = ["symbol", "entryPrice", "markPrice", "pnl", "roe", "amount"]

while True:
    for trader in traders:
        print(names[trader])
        trader_info = [trader, names[trader]]    # Write the name of the trader
        
        try:
            position_array   = get_positions(trader)     # Get the current position of the trader
            positions        = position_array["data"]["otherPositionRetList"]
            request_id       = secrets.token_urlsafe(100) # Create unique id per request
        except:
            print(f"Couldn't download position for {names[trader]}")
            continue #This is useful not to mix stuff between positions and no positions        
        
        trader_in_database = len(do_query(query=f"select * from POSITIONS where encryptedUid = '{trader}' limit 1")) > 0
        if positions: 
            if trader_in_database: # if the trader is in the database, check the delta
                
                # Get last update from trader. 
                max_timestamp     = do_query(query=f"SELECT MAX(date) as timestamp FROM POSITIONS where encryptedUid = '{trader}'")["timestamp"][0]

                # Read the last request_id based on last update
                max_request_id    = do_query(query=f"SELECT request_id FROM POSITIONS WHERE date = '{max_timestamp}' limit 1")["request_id"][0]

                # Get all the positions with that request_id
                old_positions     = do_query(query=f"SELECT * FROM POSITIONS WHERE request_id = '{max_request_id}' and status = 'OPEN' ")[["symbol","amount"]]

                # Compare the previous position with the new one, pair by pair
                new_positions = pd.DataFrame(positions)[keys]
                both          = old_positions.merge(new_positions, "outer", on ="symbol", suffixes=('_old', '_new')).fillna(0) #Filling nas (if previous or new position is na, is because is not present, hence 0)
                both["difference"] = round(both["amount_new"] - both["amount_old"],4)

                # Send a message with each signal
                for i in range(len(both)):
                    row = both.loc[i]
                    if row["difference"] != 0:     #row["markPrice"] |= 0

                        if   names[trader] == "TraderT":      chat_id = -542063577
                        elif names[trader] == "ClickHereNow": chat_id = -591308509
                        else:   chat_id = -530891051

                        status = "OPEN"
                        status = "CLOSED" if row["amount_new"] == 0
                        message = create_message(names[trader], row, status)

                        try: 
                            send_message(message, chat_id)
                            print(message)
                        except: print("Too many messages")
                        if names[trader] in ["TraderT","ADnefina"] : send_message_discord(message)
                        time.sleep(2)

            for i in range(len(positions)):
                vector = [positions[i].get(key) for key in keys]
                insert_in_table(trader_info, request_id, vector, status)
                
        if not positions:
            print(f"No positions for {names[trader]}, but still save.")
            empty    = [0,0,0,0,0,0] # Need to get the timestamp right!
            insert_in_table(trader_info, request_id, empty, "CLOSE")

        time.sleep(2)

TraderT
ADnefina
No positions for ADnefina, but still save.
ClickHereNow
CheerWindCard
BananaMan1
No positions for BananaMan1, but still save.
BigPiePlayer
No positions for BigPiePlayer, but still save.
CryptoPinoy
No positions for CryptoPinoy, but still save.
TraderT
ADnefina
No positions for ADnefina, but still save.
ClickHereNow
💰ClickHereNow 2021-09-29 15:11:33
Position change from -33132.0ADAUSDT to -31386.0ADAUSDT.
BUY 1746.0ADAUSDT at 2.1151$
Status:OPEN
CheerWindCard
BananaMan1
No positions for BananaMan1, but still save.
BigPiePlayer
No positions for BigPiePlayer, but still save.
CryptoPinoy
No positions for CryptoPinoy, but still save.
TraderT
💰TraderT 2021-09-29 15:11:48
Position change from 0.0XRPUSDT to 489601.1XRPUSDT.
BUY 489601.1XRPUSDT at 0.9827$
Status:OPEN
ADnefina
No positions for ADnefina, but still save.
ClickHereNow
CheerWindCard
BananaMan1
No positions for BananaMan1, but still save.
BigPiePlayer
No positions for BigPiePlayer, but still save.
CryptoPinoy
No posi

KeyboardInterrupt: 

In [33]:
data = do_query()
#data[data["nick"]=="TraderT"]

In [34]:
data

,id,date,encryptedUid,nick,symbol,status,entryPrice,markPrice,pnl,roe,amount,updateTimeStamp,request_id
0,1,2021-09-29 14:39:54,CCF3E0CB0AAD54D9D6B4CEC5E3E741D2,TraderT,SOLUSDT,OPEN,137.959875,137.424000,-3832.039980,-0.038994,7151.0,1632919194.48632,tZEG_QRQt6Uj9OOaTjhKJLUyZGsqLjP_qc1-1ldjCalX3A...
1,2,2021-09-29 14:39:54,CCF3E0CB0AAD54D9D6B4CEC5E3E741D2,TraderT,DYDXUSDT,OPEN,25.132195,24.785881,-3435.644057,-0.069861,9920.6,1632919194.51199,tZEG_QRQt6Uj9OOaTjhKJLUyZGsqLjP_qc1-1ldjCalX3A...
2,3,2021-09-29 14:39:56,49A7275656A7ABF56830126ACC619FEB,ADnefina,0,None,0.000000,0.000000,0.000000,0.000000,0.0,1632919196.81362,QeUHyqEfk3IZ1jWy-fC0OMTmdSyz7P2POwKLtbCAknjqvr...
3,4,2021-09-29 14:39:57,D3AFE978B3F0CD58489BC27B35906769,CheerWindCard,KAVAUSDT,OPEN,5.383570,5.258422,-173.955206,-0.050292,1390.0,1632919197.10158,LAMef7E7SALeMuGMbX_2OiGT1t4YSfyGtbUOHcvqgBK-zL...
4,5,2021-09-29 14:39:59,FB23E1A8B7E2944FAAEC6219BBDF8243,BananaMan1,0,None,0.000000,0.000000,0.000000,0.000000,0.0,1632919199.46337,qjGHOjCt9ZTiyFGt9oIZK_gnjhdfUWQhQuKNLcam-bBuc7...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
141,142,2021-09-29 14:47:25,49A7275656A7ABF56830126ACC619FEB,ADnefina,0,CLOSE,0.000000,0.000000,0.000000,0.000000,0.0,1632919645.05809,IhvXMuPPy226g3FuWfPgIUDoyncARYOfSrWorTrmuqTCiP...
142,143,2021-09-29 14:47:25,D3AFE978B3F0CD58489BC27B35906769,CheerWindCard,KAVAUSDT,OPEN,5.383570,5.276100,-149.382939,-0.043127,1390.0,1632919645.35934,Uiyp_NyJgB5ixIJ24tMJ-IZ8z0O_3fkXOOHuLd85yUVZvj...
143,144,2021-09-29 14:47:28,FB23E1A8B7E2944FAAEC6219BBDF8243,BananaMan1,0,CLOSE,0.000000,0.000000,0.000000,0.000000,0.0,1632919648.30841,O-DFI7LMJKlUyK1BIOw3cZyqD7jnhw3lL0k2pY8FUu6eIc...
144,145,2021-09-29 14:47:28,F318AE1B1BB3AEF4EDBD36E5AE250CD1,BigPiePlayer,0,CLOSE,0.000000,0.000000,0.000000,0.000000,0.0,1632919648.58731,BuCFBUhty3WI8bjr4Lr-3c3op6AiA4IaWDT51LZZyMOV0Z...
